Conecto al Drive e importo las librerías




In [ ]:
import pandas as pd
import requests
from pandas import json_normalize
from time import sleep

In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/'

Mounted at /content/gdrive
/content/gdrive/MyDrive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive




*   CONECTO AL DATASET




In [ ]:
df_Attrition = pd.read_csv('/content/drive/MyDrive/DATA SCIENCE II - PreEntrega/HR_Employee_Attrition (3).csv', sep=',')
display(df_Attrition.head())

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


Defino una función para traer datos de RandomUser

📌 Usa batches para evitar límites.

📌 Permite fijar una seed (repetible).

📌 Incluye manejo simple de reintentos.

In [ ]:
def fetch_randomuser(n_rows:int,
                     batch_size:int=1000,
                     nat:str='us,ca',
                     inc:str='location,dob,login,gender',
                     seed:str='attrition_seed',
                     max_retries:int=3,
                     pause:float=0.6) -> pd.DataFrame:
    """
    Descarga n_rows registros desde RandomUser en lotes y retorna un DataFrame normalizado.
    """
    results = []
    remaining = n_rows

    while remaining > 0:
        size = min(batch_size, remaining)
        url = (
            'https://randomuser.me/api/'
            f'?results={size}&nat={nat}&inc={inc}&seed={seed}'
        )

        for attempt in range(1, max_retries+1):
            try:
                r = requests.get(url, timeout=30)
                r.raise_for_status()
                chunk = r.json().get('results', [])
                if not chunk:
                    raise ValueError('Respuesta sin "results".')
                results.extend(chunk)
                break
            except Exception as e:
                if attempt == max_retries:
                    raise
                sleep(pause)  # espera corta y reintenta

        remaining -= size
        sleep(pause)   # cuida el rate limit

    df_api = json_normalize(results)
    return df_api

Llamo a la API y normalizo

Traigo el mismo número de filas que el dataset para hacer un merge 1:1 por índice.

In [ ]:
n = len(df_Attrition)
df_api_raw = fetch_randomuser(n_rows=n)
df_api_raw.head(), df_api_raw.shape

(   gender  location.street.number location.street.name location.city  \
 0  female                    9574            W Gray St       Saginaw   
 1  female                    9598   Concession Road 23   South River   
 2  female                    7310          E Center St    Long Beach   
 3    male                    6674            George St      Carleton   
 4    male                    1199              Plum St  Garden Grove   
 
           location.state location.country location.postcode  \
 0               Kentucky    United States             32613   
 1               Manitoba           Canada           S2G 2W4   
 2               Nebraska    United States             99117   
 3  Northwest Territories           Canada           U6W 6W9   
 4                  Idaho    United States             43448   
 
   location.coordinates.latitude location.coordinates.longitude  \
 0                      -50.6678                        57.7152   
 1                       83.0324        

Selecciono y renombro columnas útiles

📢 Añado prefijo ext_ para no chocar con nombres existentes.

In [ ]:
cols_map = {
    "location.city": "ext_city",
    "location.state": "ext_state",
    "location.postcode": "ext_postcode",
    "location.coordinates.latitude": "ext_lat",
    "location.coordinates.longitude": "ext_lon",
    "dob.age": "ext_age_api",
    "gender": "ext_gender",
}

df_api = df_api_raw[list(cols_map.keys())].rename(columns=cols_map)

# Tipos recomendados
df_api["ext_postcode"] = df_api["ext_postcode"].astype(str)
df_api["ext_lat"] = pd.to_numeric(df_api["ext_lat"], errors="coerce")
df_api["ext_lon"] = pd.to_numeric(df_api["ext_lon"], errors="coerce")

df_api.head()

,ext_city,ext_state,ext_postcode,ext_lat,ext_lon,ext_age_api,ext_gender
0,Saginaw,Kentucky,32613,-50.6678,57.7152,73,female
1,South River,Manitoba,S2G 2W4,83.0324,-33.8292,52,female
2,Long Beach,Nebraska,99117,49.9940,-169.4797,59,female
3,Carleton,Northwest Territories,U6W 6W9,57.1375,78.4214,30,male
4,Garden Grove,Idaho,43448,82.1320,-44.5369,34,male


Uno (merge) por índice 1:1

⛔ Documento que es una simulación (no existe ID real para matchear con API).

In [ ]:
df_attr = df_Attrition.reset_index(drop=True)
df_api  = df_api.reset_index(drop=True)

df_enriched = pd.concat([df_attr, df_api], axis=1)
df_enriched.head(), df_enriched.shape

(   Age Attrition     BusinessTravel  DailyRate              Department  \
 0   41       Yes      Travel_Rarely       1102                   Sales   
 1   49        No  Travel_Frequently        279  Research & Development   
 2   37       Yes      Travel_Rarely       1373  Research & Development   
 3   33        No  Travel_Frequently       1392  Research & Development   
 4   27        No      Travel_Rarely        591  Research & Development   
 
    DistanceFromHome  Education EducationField  EmployeeCount  EmployeeNumber  \
 0                 1          2  Life Sciences              1               1   
 1                 8          1  Life Sciences              1               2   
 2                 2          2          Other              1               4   
 3                 3          4  Life Sciences              1               5   
 4                 2          1        Medical              1               7   
 
    ...  YearsInCurrentRole YearsSinceLastPromotion  YearsWi

💾 Guardo el resultado

In [ ]:
out_csv = '/content/HR_Employee_Attrition_enriquecido.csv'
df_enriched.to_csv(out_csv, index=False)
out_csv, df_enriched.shape

('/content/HR_Employee_Attrition_enriquecido.csv', (2774, 42))

In [ ]:
df_enriched[["ext_city","ext_state","ext_postcode"]].isna().mean()

,0
ext_city,0.0
ext_state,0.0
ext_postcode,0.0


📌 Checklist de validación rápida

In [ ]:
df_enriched[["ext_lat","ext_lon"]].describe()

,ext_lat,ext_lon
count,2774.000000,2774.000000
mean,1.915015,2.999336
std,52.151379,102.849287
min,-89.945100,-179.912000
25%,-43.879100,-87.741975
50%,3.507700,5.336750
75%,46.626100,91.964900
max,89.502700,179.873500


In [ ]:
set(df_enriched.columns) - set(df_attr.columns)

{'ext_age_api',
 'ext_city',
 'ext_gender',
 'ext_lat',
 'ext_lon',
 'ext_postcode',
 'ext_state'}

Data dictionary (columnas nuevas) =

✅ ext_city: Ciudad proveniente de RandomUser (texto).

✅ ext_state: Estado/provincia (texto).

✅ ext_postcode: Código postal (string).

✅ ext_lat / ext_lon: Coordenadas aproximadas de la ubicación (float).

✅ ext_age_api: Edad calculada por la API (int).

✅ ext_gender: Género provisto por la API (texto, “male/female”).

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
from datetime import datetime


carpeta = '/content/drive/MyDrive/DATA SCIENCE II - PreEntrega/' # Corrected to be the directory path

archivo = f'HR_Employee_Attrition_enriquecido_{datetime.now():%Y%m%d_%H%M}.csv'

ruta_salida = os.path.join(carpeta, archivo)


df_enriched.to_csv(ruta_salida, index=False)

print('Guardado en:', ruta_salida)

Mounted at /content/gdrive


OSError: [Errno 107] Transport endpoint is not connected: '/content/drive/MyDrive/DATA SCIENCE II - PreEntrega'